# Test Vector Generator for COBS Decoder VHDL Entity

The COBS decoder is a VHDL entity that takes a single AXI-Stream input, one byte wide, containing a stream of bytes encoded according to COBS, the Consistent Overhead Byte Stuffing algorithm detailed in [this IEEE/ACM Transactions on Networking paper](http://www.stuartcheshire.org/papers/cobsforton.pdf). It generates a single AXI-Stream output, also one byte wide, containing the decoded data from the input stream, and broken up into frames according to the rules of COBS.

The complexity of the COBS decoder entity is mostly due to the handshaking requirements. The AXI-S input and output are designed to flow through data as it is processed. Either the data producer on the input or the data consumer on the output can require a delay in transferring the next byte. Each entity is responsible for passing through this "back pressure" from its output to its input, so that data is not allowed to back up without limit anywhere in the system. Both interfaces share a common clock `clk` and reset `rst`.

On the input side, `s_tdata` is the byte-wide data, `s_tvalid` indicates on which rising edges of `clk` the data and other signals are valid, and `s_tready` (an output signal) indicates on which rising edges the entity is able to accept a data transfer. An additional input-related signal, `s_tlast`, indicates the last byte of a frame on the input. We rely instead on the COBS framing to break up the input stream into frames, so we ignore `s_tlast` entirely.

On the output side, `m_data` is the byte-wide data, and `m_tvalid`, `m_tready`, and `m_tlast` have the corresponding meanings. The COBS decoder entity asserts `m_tlast` on the last byte of a frame as derived from the COBS framing information.

We use some specific knowledge about the COBS decoder implementation to create test vectors for it.

1. The COBS decoder inserts two clocks of delay between the input stream and the output stream, when both handshaking inputs (`s_tvalid` and `m_tready`) are high.
2. The COBS decoder passes `m_tready` through to `s_tready` without any clock delays.

The test vector file is constructed out of valid COBS frames, each of which starts and ends with the frame separator byte, a zero. These frames have randomly chosen lengths (within a specified range) and randomly generated contents. With some probability, garbage bytes are inserted between frames to challenge the error recovery capability of the decoder. Once this byte stream is constructed, a random set of `s_tvalid` and `m_tready` deassertion events is generated and integrated with the byte stream.
* when `s_tvalid` is low (but `m_tready` is high) the meaning in AXI-S is that the bytes on `s_tdata` are not valid, and the arrival of the next valid input byte is delayed. To represent this, and to test the decoder's rejection of these invalid bytes, we fill `s_tdata` with random bytes.
* when `m_tready` is low, and thus the decoder is driving `s_tready` low (but `s_tvalid` is high) the meaning in AXI-S is that the first byte offered on `s_tdata` is held steady throughout the delay, with the actual transfer not occurring until after `s_tready` is high again. To represent this accurately, we fill `s_tdata` with copies of the next byte from the data stream.
* when `s_tvalid` is low and `s_tready` goes from high to low, the fill data remains random unless and until `s_tvalid` goes high. If `s_tready` is still low by then, the next single byte is clocked in and held until it can be clocked in, when `s_tready` is high.
* when `s_tready` is low, a transition of `s_tvalid` from high to low is not permitted.

By following all these rules, this program can generate cycle-by-cycle test vectors that correspond to the intended operation of the COBS decoder entity in an AXI-S to AXI-S instantiation. These vectors, which include `s_tdata`, `s_tvalid`, `m_tready`, and `rst`, are output to a plain text file suitable for importing into a Vivado simulation environment.

The program also generates a file containing the data structure of original random unencoded frames. A separate program extracts the decoded frames from the simulation results output file, and compares them.

Requires Python >= 3.9 (because of random.randbytes() at least)

Developed with Python 3.11.0

In [ ]:
from cobs import cobs   # https://github.com/cmcqueen/cobs-python
import pickle
import random

In [ ]:
# Parameters for test vector generation
# These may need to be turned into command line arguments for convenience.

tv_number_of_frames = 1000
tv_min_frame_length = 1             # bytes
tv_max_frame_length = 275           # bytes; not realistically long, but bigger than the max COBS sequence
# tv_max_frame_length = 10000         # bytes; bigger than the max DVB-S2 BBFRAME length
tv_garbage_probability = 0.0        # probability per frame boundary of garbage insertion (0.0 for sim-compare-frames to work)
tv_min_garbage_length = 1           # bytes
tv_max_garbage_length = 30          # bytes
tv_s_tvalid_low_probability = 0.1   # probability per byte of s_tvalid deassertion
tv_s_tvalid_max_cycles = 10         # maximum length of s_tvalid deassertion
tv_m_tready_low_probability = 0.1   # probability per byte of m_tready deassertion
tv_m_tready_max_cycles = 10         # maximum length of m_tready deassertion

In [ ]:
# Seed the random number generator. The seed becomes a filename tag for reproducibility.
seed = random.getrandbits(32)
# seed = 0xe6f1308e  # override seed to regenerate a previous run
random.seed(seed)

seed_tag = f"{seed:08x}"
tv_frames_filename = f"COBS_dec-tv-frames-{seed_tag}"
tv_vectors_filename = f"COBS_dec-tv-in-{seed_tag}"


In [ ]:
# Build a list of random unencoded frames

unencoded_frames = []
for _ in range(0, tv_number_of_frames):
    length = random.randint(tv_min_frame_length, tv_max_frame_length)
    data = random.randbytes(length)
    unencoded_frames += [data]
    print(f"Encoded frame length={length}\n{data.hex(' ')}")


In [ ]:
# Save the original random unencoded frames for comparison with simulation results

with open(tv_frames_filename, "wb") as f:
    pickle.dump(unencoded_frames, f)

In [ ]:
# Build encoded data stream (before handshaking)

encoded_bytes = bytearray()

for frame in unencoded_frames:
    encoded_bytes += b"\0"
    encoded_frame = cobs.encode(frame)
    encoded_bytes += encoded_frame

    print(f"COBS frame length={len(encoded_frame)} count={encoded_frame[0]} \n{encoded_frame.hex(' ')}")

    if random.random() < tv_garbage_probability:
        garbage_length = random.randint(tv_min_garbage_length, tv_max_garbage_length)
        garbage_bytes = random.randbytes(garbage_length)
        encoded_bytes += garbage_bytes
        print(f"Garbage length={garbage_length}\n{garbage_bytes.hex(' ')}")

encoded_bytes += b"\0"


In [ ]:
# Build encoded data stream with handshaking

# These will be our eventual test vector outputs
s_tdata = bytearray()   # copy of the encoded data stream, with handshaking adjustments
s_tvalid = bytearray()  # value of the s_tvalid signal as generated
m_tready = bytearray()  # value of the m_tready (== s_tready) signal as generated

# counts of remaining low cycles of handshaking signals
s_tvalid_count = 0
m_tready_count = 0

# first byte (a zero) gets a pass on handshaking
s_tdata += b"\0"
s_tvalid += bytes([1])
m_tready += bytes([1])

# Now pass through all the encoded bytes, inserting handshaking as generated
for ebyte in encoded_bytes[1:]:
    # if handshaking not already happening, see if it needs to start on each signal:
    if s_tvalid_count == 0 and m_tready_count == 0:
        if random.random() < tv_s_tvalid_low_probability:
            s_tvalid_count = random.randint(1, tv_s_tvalid_max_cycles)
    if m_tready_count == 0:
        if random.random() < tv_m_tready_low_probability:
            m_tready_count = random.randint(1, tv_m_tready_max_cycles)

    # insert cycles as long as handshaking line(s) deasserted
    while s_tvalid_count > 0 or m_tready_count > 0:
        if s_tvalid_count > 0:
            s_tvalid_count -= 1
            s_tdata += random.randbytes(1)  # with s_tvalid low, s_tdata is indeterminate
            s_tvalid += b"\0"
            m_tready += b"\0" if m_tready_count > 0 else b"\1"
        else:  # m_tready_count > 0
            m_tready_count -= 1
            s_tdata += bytes([ebyte])  # with s_tready low but s_tvalid high, s_tdata freezes
            s_tvalid += b"\1"
            m_tready += b"\0"

    # handshaking signals are both high now, carry on
    s_tdata += bytes([ebyte])
    s_tvalid += b"\1"
    m_tready += b"\1"

# a few cycles for things to settle down a bit
s_tdata += b"\0"
s_tvalid += bytes([1])
m_tready += bytes([1])

s_tdata += b"\0"
s_tvalid += bytes([1])
m_tready += bytes([1])

s_tdata += b"\0"
s_tvalid += bytes([1])
m_tready += bytes([1])

In [ ]:
# Now emit the test vectors file
with open(tv_vectors_filename, "w") as f:
    print("0 0 0 1", file=f)  # start with a reset cycle
    for a, b, c in zip(s_tdata, s_tvalid, m_tready):
        print(a, b, c, 0, file=f)
